In [1]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import json, os, pprint, time
from urllib import parse

In [2]:
options = webdriver.ChromeOptions()
# 視窗最大
options.add_argument('--start-maximized')
# 無痕
options.add_argument('--incongito')
# 防跳出視窗
options.add_argument('--disable')


In [20]:
driver = webdriver.Chrome(options = options)

listData = []

url = 'https://www.bookwormzz.com/zh/'

def visit():
    driver.get(url)

def getMainLinks():
    aLinks = driver.find_elements(By.CSS_SELECTOR, "a[data-ajax='false']")
    for a in aLinks:
        listData.append({
            "title": a.get_attribute('innerText'),
            "link": parse.unquote(a.get_attribute('href'))+"#book_toc"
        })
def getSubLinks():
    for i in range(len(listData)):
        if "sub" not in listData[i]:
            listData[i]["sub"] = []
        driver.get(listData[i]["link"])
        try:
            WebDriverWait(driver, 3).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "a[rel='external'][class='ui-link']")
                )
            )
            
            aLinks = driver.find_elements(By.CSS_SELECTOR, "a[rel='external'][class='ui-link']")
            
            for a in aLinks:
                listData[i]["sub"].append({
                    "sub_title": a.get_attribute("innerText"),
                    "sub_link": parse.unquote( a.get_attribute('href'))
                })
            
        except TimeoutException as e:
            continue
    
def saveJson():
    f = open("JinYong.json","w",encoding="utf-8")
    f.write( json.dumps(listData, ensure_ascii=False))
    f.close()

def saveTxt():
    listContent = []
    
    f = open("JinYong.json","r",encoding="utf-8")
    strJson = f.read()
    f.close()
    
    folderPath = '/Users/gaoming/Downloads/JinYong'
    if not os.path.exists(folderPath):
        os.makedirs(folderPath)
    
    listResult = json.loads(strJson)
    for i in range(len(listResult)):
        for j in range(len(listResult[i]["sub"])):
            driver.get(listResult[i]["sub"][j]["sub_link"])
            div = driver.find_element(By.CSS_SELECTOR, 'div#html > div')
            strContent = div.get_attribute('innerText')
            strContent = strContent.replace(" ","")
            strContent = strContent.replace("\r","")
            strContent = strContent.replace("\n","")
            
            fileName = f"{listResult[i]['title']}_{listResult[i]['sub'][j]['sub_title']}.txt"
            fp = open(f"{folderPath}/{fileName}","w",encoding="utf-8")
            fp.write(strContent)
            fp.close()
            
            listContent.append(strContent)
            
    f = open("train.json","w",encoding="utf-8")
    f.write(json.dumps(listContent, ensure_ascii=False))
    f.close()
    
def close():
    driver.quit()
    
    
        
if __name__ == "__main__":
    time1 = time.time()
    visit()
    getMainLinks()
    getSubLinks()
    saveJson()
    saveTxt()
    close()
    print(f"[total] 花了 {time.time()-time1}秒")

[total] 花了 768.6458439826965秒
